# Assignment Data Structure and Solution
#### We can use this Jupyter Notebook to analysis the data structure of different file and variables to help us to solve these three model construction and analysis.

# First Class(CommandLine)
<big>**Calss Function**</big>:
   1. Judge the command line to accord with the stationary Formate.
   2. Print Help Inforamtion if the command formate is wrong.
   3. Default:<br>
      1)'index_nostoplist_nostemming.txt' / queries_nostoplist_nostemming.txt'. <br>
      2) For the First dataset(index?), it's the key vocabularly in the text. <br>
      3) For the second dataset(queries?), it's the summary of all the search queris. <br>
       
**Example Command Formate**:
```
> python ir_engine.py -o result.txt
```

In [9]:
#==============================================================================
# Importing
import sys, getopt, re
import time
#==============================================================================
# Command line processing
class CommandLine:
    def __init__(self):
        opts, args = getopt.getopt(sys.argv[1:], 'hspw:o:')
        opts = dict(opts)
        self.exit = True

        if '-h' in opts:
            self.printHelp()
            return

        if len(args) > 0:
            print("*** ERROR: no arg files - only options! ***", file=sys.stderr)
            self.printHelp()
            return

        if '-w' in opts:
            if opts['-w'] in ('binary', 'tf', 'tfidf'):
                self.termWeighting = opts['-w']
            else:
                warning = (
                    "*** ERROR: term weighting label (opt: -w LABEL)! ***\n"
                    "    -- value (%s) not recognised!\n"
                    "    -- must be one of: binary / tf / tfidf"
                    )  % (opts['-w'])
                print(warning, file=sys.stderr)
                self.printHelp()
                return
        else:
            self.termWeighting = 'binary'

        if '-o' in opts:
            self.outfile = opts['-o']
        else:
            print("*** ERROR: must specify output file (opt: -o FILE) ***",
                  file=sys.stderr)
            self.printHelp()
            return

        if '-s' in opts and '-p' in opts:
            self.indexFile   = 'index_withstoplist_withstemming.txt'
            self.queriesFile = 'queries_withstoplist_withstemming.txt'
        elif '-s' in opts:
            self.indexFile   = 'index_withstoplist_nostemming.txt'
            self.queriesFile = 'queries_withstoplist_nostemming.txt'
        elif '-p' in opts:
            self.indexFile   = 'index_nostoplist_withstemming.txt'
            self.queriesFile = 'queries_nostoplist_withstemming.txt'
        else:
            self.indexFile   = 'index_nostoplist_nostemming.txt'
            self.queriesFile = 'queries_nostoplist_nostemming.txt'
            
        self.exit = False

    def printHelp(self):
        progname = sys.argv[0]
        progname = progname.split('/')[-1] # strip off extended path
        help = __doc__.replace('<PROGNAME>', progname, 1)
        print(help, file=sys.stderr)


# Second Class(IndexLoader)
<big>**Calss Function**</big>:
   1. Use the **Regular Expression** to extract the information from original file and index in a new dictionary.
   2. The New Generator dictionary has 2 dimensions.

<big>**Data Structure(Python Dictionary)**</big>:<br>
   1. {(term name)->{(term name)-> Count}}

In [10]:
#==============================================================================
# Load (precomputed) Index File for (preprocessed) Document Collection
class IndexLoader:
    def __init__(self, indexFile):
        self.index = {}
        docidCountRE = re.compile('(\d+):(\d+)')
        f = open(indexFile, 'r')
        for line in f:
            term = line.split(' ', 1)[0]
            self.index[term] = {}
            for (docid, count) in docidCountRE.findall(line):
                docid = int(docid)
                self.index[term][docid] = int(count)
    def getIndex(self):
        return self.index

# Third Class(Queries)
<big>**Calss Function**</big>:
   1. Use the **Regular Expression** to extract the information from original file and index in a new dictionary.
   2. The New Generator dictionary has 2 or more dimensions.
   3. This class is to extract the queries and get the same dataset formate(python dictionary) as IndexLoader

<big>**Data Structure(Dictionary)**</big>:<br>
   1. {(qid name)->{(qid name)-> count_num}}

In [11]:
class Queries:
    def __init__(self, queriesFile):
        self.qStore = {}
        termCountRE = re.compile('(\w+):(\d+)')
        f = open(queriesFile, 'r')
        for line in f:
            qid = int(line.split(' ', 1)[0])
            self.qStore[qid] = {}
            for (term, count) in termCountRE.findall(line):
                self.qStore[qid][term] = int(count)
    
    def getQuery(self, qid):
        if qid in self.qStore:
            return self.qStore[qid]
        else:
            print("*** ERROR: unknown query identifier (\"%s\") ***" % qid, file=sys.stderr)
            if type(qid) == type(''):
                print('WARNING: query identifiers should be of type: integer', file=sys.stderr)
                print('         -- your query identifier is of type: string', file=sys.stderr)
            print(' -- program exiting', file=sys.stderr)
    
    def qids(self):
        return sorted(self.qStore)

# Forth Class(ResultStore)
<big>**Calss Function**</big>:<br>
<big>**Data Structure(Dictionary)**</big>:<br>

In [12]:
class ResultStore:
    def __init__(self, outfile):
        self.outfile = outfile
        self.results = []

    def store(self, qid, docids):
        if len(docids) > 10:
            docids = docids[:10]
        self.results.append((qid, docids))

    def output(self):
        with open(self.outfile, 'w') as out:
            for (qid, docids) in self.results:
                for docid in docids:
                    print(qid, docid, file=out)

# Target Class(Retrieve)

In [5]:
# TODO: Finish the Assignment Part
class Retrieve:
    # Define all the usable class variables
    index = None
    termWeighting = None
#     IDF_value = dict()  # IDF values for each documents
    doc_len = dict()  # The dictionary to record all the documents' length
    tr_candidate = dict()  # Retrieval all the documents 
    tr_candidate_id = list()  # Retrial all the documents ID and store for use binary model candidate
    tr_query_count = list()  # To store the query count

    # Create new Retrieve object storing index and termWeighting scheme
    def __init__(self, index, termWeighting):
        self.index = index
        self.termWeighting = termWeighting
        # get the length of the each documents and
        self.doc_length()

    # Firstly we should calculate the documents length by number of arisen vocabularly
    def doc_length(self):
        # Extract all the items from the index file and sum the arisen number
        tempor_list = self.index.items()
        for term,combination in tempor_list:
            for doc_id,show_count in combination:
                if doc_id in self.doc_len:
                    self.doc_len[doc_id] = self.doc_len[doc_id] + int(show_count)
                else:
                    self.doc_len[doc_id] = int(show_count)

    # Method performing retrieval for specified query
    def forQuery(self, query):
        # Compare the index and query
        self.getCandidate(index, query) 
        # Firstly we should judge the function the program operate
        if self.termWeighting is 'binary':
            self.binary_model(query,self.tr_document_id)
        # elif self.termWeighting is 'tfidf':

    # Extract the query from the file and also generate a new corresponding dictionary
    # We delete all the irreverent values and integrate the dictionary into a list(Count all the count)
    def getCandidate(self, index, query):
        # Get the two list to store the term of query and number of key vocabularly
        queryIterm = list(query.keys())
        qtermCount = list(query.values())
        for each_iterm in set(queryIterm):            
            if each_iterm in index:   # If the target document includes the query key we should put the id into a document
                self.tr_candidate[each_iterm] = index[each_iterm] # 
                doc_id = list(index[each_iterm].keys())
                count = list(index[each_iterm].values())
                self.tr_candidate_id.extend(doc_id)

    def binary_model(self,query,documentid):
        file_len = self.doc_len
        Results = dict()
        # Calculate all the value from the self.tr_documentid and convert it into the set to improve the efficiency

<big>**Below Code is the Formate of Command(Please customize with on the basis of your requirements)**</big><br><br>

**How to generate command line As you wish**<br>

`sys.argv` is the python reserved variables and applied to `ArgumentParser.parse_args(args=None, namespace=None)` to be the one of the parameters. This is a list of strings, each of which is obviously derived from command line arguments, separated by spaces. We can redefine this list by below command.

We should delete the key parameters like `python` and put the remains into the list.

In [6]:
# Example Command Formate:
#  > python ir_engine.py -o result.txt
sys.argv = ['ir_engine.py', '-o', 'result.txt']

In [38]:
type(queries.getQuery(1).items())

dict_items

In [51]:
queryIterm, qtermCount = zip(*queries.getQuery(1).items())
queryIterm 

('an',
 'articles',
 'computers',
 'deal',
 'exist',
 'for',
 'ibm',
 'operating',
 'sharing',
 'system',
 'time',
 'tss',
 'what',
 'which',
 'with')

In [28]:
queries.qStore

{1: {'an': 1,
  'articles': 1,
  'computers': 1,
  'deal': 1,
  'exist': 1,
  'for': 1,
  'ibm': 1,
  'operating': 1,
  'sharing': 1,
  'system': 2,
  'time': 1,
  'tss': 1,
  'what': 1,
  'which': 1,
  'with': 1},
 2: {'am': 1,
  'articles': 1,
  'b': 1,
  'by': 1,
  'either': 1,
  'i': 1,
  'in': 1,
  'interested': 1,
  'or': 1,
  'pooch': 2,
  'prieve': 2,
  'u': 1,
  'udo': 1,
  'written': 1},
 3: {'compilers': 1,
  'construction': 1,
  'in': 1,
  'intermediate': 1,
  'languages': 1,
  'multi': 1,
  'of': 1,
  'targeted': 1,
  'tcoll': 1,
  'used': 1},
 4: {'a': 1,
  'abstract': 1,
  'and': 1,
  'are': 1,
  'as': 1,
  'between': 1,
  'but': 1,
  'calls': 1,
  'communicating': 1,
  'complete': 1,
  'descriptions': 1,
  'disjoint': 1,
  'distributed': 1,
  'environment': 1,
  'examples': 1,
  'exclusively': 1,
  'for': 1,
  'i': 2,
  'implementations': 1,
  'in': 2,
  'interested': 1,
  'interests': 1,
  'm': 1,
  'mechanisms': 2,
  'message': 1,
  'my': 1,
  'not': 1,
  'of': 2,
  '

In [19]:
config = CommandLine()
index = IndexLoader(config.indexFile).getIndex()
retrieve = Retrieve(index, config.termWeighting)
queries = Queries(config.queriesFile)
allResults = ResultStore(config.outfile)

In [ ]:
for qid in queries.qids():
    query = queries.getQuery(qid)
    results = retrieve.forQuery(query)
    allResults.store(qid, results)

allResults.output()